In [712]:
import torch
import numpy as np
from torch.distributions import normal
from torch.distributions import multivariate_normal
import csv
import pandas as pd
import math


df1 = pd.read_csv("nba2021_per_game.csv")
df_adv1 = pd.read_csv("nba2021_advanced.csv")
df1

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,Precious Achiuwa,PF,21,MIA,28,2,14.6,2.6,4.4,0.590,...,0.561,1.3,2.7,4.0,0.6,0.4,0.5,1.0,1.9,6.5
1,Jaylen Adams,PG,24,MIL,6,0,2.8,0.2,1.3,0.125,...,0.000,0.0,0.5,0.5,0.3,0.0,0.0,0.0,0.2,0.3
2,Steven Adams,C,27,NOP,27,27,28.1,3.5,5.8,0.603,...,0.468,4.3,4.6,8.9,2.1,1.0,0.6,1.7,1.9,8.0
3,Bam Adebayo,C,23,MIA,26,26,33.6,7.4,12.9,0.573,...,0.841,1.9,7.3,9.2,5.3,1.0,1.0,3.0,2.6,19.9
4,LaMarcus Aldridge,C,35,SAS,18,18,26.7,5.9,12.5,0.476,...,0.762,0.8,3.5,4.3,1.9,0.4,0.9,0.9,1.5,14.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
492,Delon Wright,SG,28,DET,28,23,29.4,3.7,8.1,0.456,...,0.785,1.1,3.5,4.6,4.9,1.4,0.5,1.2,1.1,10.5
493,Thaddeus Young,PF,32,CHI,23,0,25.7,5.1,8.7,0.587,...,0.500,2.4,3.4,5.8,4.4,1.4,0.5,2.0,2.9,11.2
494,Trae Young,PG,22,ATL,26,26,34.7,7.6,17.7,0.429,...,0.887,0.7,3.2,3.8,9.3,0.8,0.3,4.3,2.0,26.5
495,Cody Zeller,C,28,CHO,15,12,24.1,4.2,8.0,0.525,...,0.759,2.6,5.4,8.0,2.5,0.7,0.6,0.9,2.5,10.1


In [713]:
df1 = df1.drop([ 'ORB', 'DRB'], axis=1)
TOT_i = [8, 181, 255 , 265, 354, 382, 403, 425]
df1 = df1.drop(TOT_i)
df1['PER'] = df_adv1['PER']
df1['TS%'] = df_adv1['TS%']

df1.at[117,'Player'] = "Luka Doncic"
df1.at[237,'Player'] = "Nikola Jokic"
df1.at[462,'Player'] = "Nikola Vucevic"

team_records = {'ATL':0.569 , "BRK":0.667 , "BOS":0.5 , "CHO":0.458 , "CHI":0.431 , "CLE":0.306 , "DAL":0.583 , 
                "DEN":0.653 , "DET":0.278 , "GSW":0.542 , "HOU":0.236 , "IND":0.427 , "LAC":0.653 , "LAL":0.583 , 
                "MEM":0.528 , "MIA":0.556 , "MIL":0.639 , "MIN":0.319 , "NOP":0.431 , "NYK":0.569 , "OKC":0.306 , 
                "ORL":0.292 , "PHI":0.681 , "PHO":0.708 , "POR":0.583 , "SAC":0.431 , "SAS":0.458 , 
                "TOR":0.375 , "UTA":0.722 , "WAS":0.427}

player_team_records = []

for team in df1["Tm"]:
    player_team_records.append(team_records[team])

df1["Team Win%"] = player_team_records
df1

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,TRB,AST,STL,BLK,TOV,PF,PTS,PER,TS%,Team Win%
0,Precious Achiuwa,PF,21,MIA,28,2,14.6,2.6,4.4,0.590,...,4.0,0.6,0.4,0.5,1.0,1.9,6.5,15.1,0.599,0.556
1,Jaylen Adams,PG,24,MIL,6,0,2.8,0.2,1.3,0.125,...,0.5,0.3,0.0,0.0,0.0,0.2,0.3,-6.9,0.125,0.639
2,Steven Adams,C,27,NOP,27,27,28.1,3.5,5.8,0.603,...,8.9,2.1,1.0,0.6,1.7,1.9,8.0,15.9,0.592,0.431
3,Bam Adebayo,C,23,MIA,26,26,33.6,7.4,12.9,0.573,...,9.2,5.3,1.0,1.0,3.0,2.6,19.9,22.7,0.641,0.556
4,LaMarcus Aldridge,C,35,SAS,18,18,26.7,5.9,12.5,0.476,...,4.3,1.9,0.4,0.9,0.9,1.5,14.1,15.2,0.542,0.458
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
492,Delon Wright,SG,28,DET,28,23,29.4,3.7,8.1,0.456,...,4.6,4.9,1.4,0.5,1.2,1.1,10.5,16.6,0.559,0.278
493,Thaddeus Young,PF,32,CHI,23,0,25.7,5.1,8.7,0.587,...,5.8,4.4,1.4,0.5,2.0,2.9,11.2,18.5,0.597,0.431
494,Trae Young,PG,22,ATL,26,26,34.7,7.6,17.7,0.429,...,3.8,9.3,0.8,0.3,4.3,2.0,26.5,22.9,0.598,0.569
495,Cody Zeller,C,28,CHO,15,12,24.1,4.2,8.0,0.525,...,8.0,2.5,0.7,0.6,0.9,2.5,10.1,18.2,0.569,0.458


In [714]:
all_stars_2021 = ["LeBron James", "Kevin Durant", "Anthony Davis", "Kawhi Leonard", "Luka Doncic", "James Harden", "Nikola Jokic",
            "Damian Lillard", "Chris Paul", "Domantas Sabonis", "Ben Simmons", "Jayson Tatum",
            "Stephen Curry", "Jaylen Brown", "Paul George", "Rudy Gobert", "Joel Embiid", "Devin Booker", 
             "Nikola Vucevic", "Julius Randle", "Donovan Mitchell", "Zach LaVine", "Mike Conley", "Zion Williamson",
            "Bradley Beal", "Kyrie Irving", "Giannis Antetokounmpo"]
is_allstar_2021 = []

for player in df1['Player']:
    if player in all_stars_2021:
        is_allstar_2021.append(1)
    else:
        is_allstar_2021.append(0)
        
df1["all star"] = is_allstar_2021

print("# of all stars 2021 = ", len(all_stars_2021))

df1 = df1.drop(['Tm', 'Pos'], axis=1)

df1

# of all stars 2021 =  27


,Player,Age,G,GS,MP,FG,FGA,FG%,3P,3PA,...,AST,STL,BLK,TOV,PF,PTS,PER,TS%,Team Win%,all star
0,Precious Achiuwa,21,28,2,14.6,2.6,4.4,0.590,0.0,0.0,...,0.6,0.4,0.5,1.0,1.9,6.5,15.1,0.599,0.556,0
1,Jaylen Adams,24,6,0,2.8,0.2,1.3,0.125,0.0,0.3,...,0.3,0.0,0.0,0.0,0.2,0.3,-6.9,0.125,0.639,0
2,Steven Adams,27,27,27,28.1,3.5,5.8,0.603,0.0,0.0,...,2.1,1.0,0.6,1.7,1.9,8.0,15.9,0.592,0.431,0
3,Bam Adebayo,23,26,26,33.6,7.4,12.9,0.573,0.1,0.2,...,5.3,1.0,1.0,3.0,2.6,19.9,22.7,0.641,0.556,0
4,LaMarcus Aldridge,35,18,18,26.7,5.9,12.5,0.476,1.3,3.7,...,1.9,0.4,0.9,0.9,1.5,14.1,15.2,0.542,0.458,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
492,Delon Wright,28,28,23,29.4,3.7,8.1,0.456,0.9,2.4,...,4.9,1.4,0.5,1.2,1.1,10.5,16.6,0.559,0.278,0
493,Thaddeus Young,32,23,0,25.7,5.1,8.7,0.587,0.2,0.9,...,4.4,1.4,0.5,2.0,2.9,11.2,18.5,0.597,0.431,0
494,Trae Young,22,26,26,34.7,7.6,17.7,0.429,2.3,6.2,...,9.3,0.8,0.3,4.3,2.0,26.5,22.9,0.598,0.569,0
495,Cody Zeller,28,15,12,24.1,4.2,8.0,0.525,0.2,1.1,...,2.5,0.7,0.6,0.9,2.5,10.1,18.2,0.569,0.458,0


In [715]:
training_vec = []
training_lab = []

for x, row in df1.iterrows():
    t_vec = []
    label = int(row[28])
    training_lab.append(label)
    for i in range(len(row)):
        data = row[i]
        #print("data = ", data)
        if i != len(row)-1 and i != 0:
            t_vec.append(data)
    training_vec.append(t_vec)

train_vec = torch.tensor(training_vec)
train_lab = torch.tensor(training_lab)
print("train_vec[0] = ", train_vec[0])
print("size^ = ", len(train_vec[0]))
print("size train_vec = ", len(train_vec))
print(train_lab[0])

train_vec[0] =  tensor([21.0000, 28.0000,  2.0000, 14.6000,  2.6000,  4.4000,  0.5900,  0.0000,
         0.0000,  0.0000,  2.6000,  4.4000,  0.5900,  0.5900,  1.3000,  2.4000,
         0.5610,  4.0000,  0.6000,  0.4000,  0.5000,  1.0000,  1.9000,  6.5000,
        15.1000,  0.5990,  0.5560])
size^ =  27
size train_vec =  489
tensor(0)


In [716]:
df2 = pd.read_csv("nba2020_per_game.csv")
df_adv2 = pd.read_csv("nba2020_advanced.csv")
df2

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,Steven Adams,C,26,OKC,63,63,26.7,4.5,7.6,0.592,...,0.582,3.3,6.0,9.3,2.3,0.8,1.1,1.5,1.9,10.9
1,Bam Adebayo,PF,22,MIA,72,72,33.6,6.1,11.0,0.557,...,0.691,2.4,7.8,10.2,5.1,1.1,1.3,2.8,2.5,15.9
2,LaMarcus Aldridge,C,34,SAS,53,53,33.1,7.4,15.0,0.493,...,0.827,1.9,5.5,7.4,2.4,0.7,1.6,1.4,2.4,18.9
3,Kyle Alexander,PF,23,MIA,2,0,6.5,0.5,1.0,0.500,...,NaN,1.0,0.5,1.5,0.0,0.0,0.0,0.5,0.5,1.0
4,Nickeil Alexander-Walker,SG,21,NOP,47,1,12.6,2.1,5.7,0.368,...,0.676,0.2,1.6,1.8,1.9,0.4,0.2,1.1,1.2,5.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
646,Trae Young,PG,21,ATL,60,60,35.3,9.1,20.8,0.437,...,0.860,0.5,3.7,4.3,9.3,1.1,0.1,4.8,1.7,29.6
647,Cody Zeller,C,27,CHO,58,39,23.1,4.3,8.3,0.524,...,0.682,2.8,4.3,7.1,1.5,0.7,0.4,1.3,2.4,11.1
648,Tyler Zeller,C,30,SAS,2,0,2.0,0.5,2.0,0.250,...,NaN,1.5,0.5,2.0,0.0,0.0,0.0,0.0,0.0,1.0
649,Ante Žižić,C,23,CLE,22,0,10.0,1.9,3.3,0.569,...,0.737,0.8,2.2,3.0,0.3,0.3,0.2,0.5,1.2,4.4


In [717]:
df2 = df2.drop([ 'ORB', 'DRB'], axis=1)
df2['PER'] = df_adv2['PER']
df2['TS%'] = df_adv2['TS%']

df2.at[177,'Player'] = "Luka Doncic"
df2.at[334,'Player'] = "Nikola Jokic"
df2.at[601,'Player'] = "Nikola Vucevic"

TOT_indicies = []
for i in range(len(df2['Tm'])):
    if df2['Tm'][i] == "TOT":
        TOT_indicies.append(i)

df2 = df2.drop(TOT_indicies)
df2

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,FT%,TRB,AST,STL,BLK,TOV,PF,PTS,PER,TS%
0,Steven Adams,C,26,OKC,63,63,26.7,4.5,7.6,0.592,...,0.582,9.3,2.3,0.8,1.1,1.5,1.9,10.9,20.5,0.604
1,Bam Adebayo,PF,22,MIA,72,72,33.6,6.1,11.0,0.557,...,0.691,10.2,5.1,1.1,1.3,2.8,2.5,15.9,20.3,0.598
2,LaMarcus Aldridge,C,34,SAS,53,53,33.1,7.4,15.0,0.493,...,0.827,7.4,2.4,0.7,1.6,1.4,2.4,18.9,19.7,0.571
3,Kyle Alexander,PF,23,MIA,2,0,6.5,0.5,1.0,0.500,...,NaN,1.5,0.0,0.0,0.0,0.5,0.5,1.0,4.7,0.500
4,Nickeil Alexander-Walker,SG,21,NOP,47,1,12.6,2.1,5.7,0.368,...,0.676,1.8,1.9,0.4,0.2,1.1,1.2,5.7,8.9,0.473
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
646,Trae Young,PG,21,ATL,60,60,35.3,9.1,20.8,0.437,...,0.860,4.3,9.3,1.1,0.1,4.8,1.7,29.6,23.9,0.595
647,Cody Zeller,C,27,CHO,58,39,23.1,4.3,8.3,0.524,...,0.682,7.1,1.5,0.7,0.4,1.3,2.4,11.1,18.8,0.576
648,Tyler Zeller,C,30,SAS,2,0,2.0,0.5,2.0,0.250,...,NaN,2.0,0.0,0.0,0.0,0.0,0.0,1.0,22.4,0.250
649,Ante Žižić,C,23,CLE,22,0,10.0,1.9,3.3,0.569,...,0.737,3.0,0.3,0.3,0.2,0.5,1.2,4.4,16.4,0.597


In [718]:
team_records = {'ATL':0.299 , "BRK":0.486 , "BOS":0.667 , "CHO":0.354 , "CHI":0.338 , "CLE":0.292 , "DAL":0.573 , 
                "DEN":0.630 , "DET":0.303 , "GSW":0.231 , "HOU":0.611 , "IND":0.616 , "LAC":0.681 , "LAL":0.732 , 
                "MEM":0.466 , "MIA":0.603 , "MIL":0.767 , "MIN":0.297 , "NOP":0.417 , "NYK":0.318 , "OKC":0.611 , 
                "ORL":0.292 , "PHI":0.681 , "PHO":0.708 , "POR":0.583 , "SAC":0.431 , "SAS":0.458 , 
                "TOR":0.736 , "UTA":0.611 , "WAS":0.347}

player_team_records = []

for team in df2["Tm"]:
    player_team_records.append(team_records[team])

df2["Team Win%"] = player_team_records

df2

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,TRB,AST,STL,BLK,TOV,PF,PTS,PER,TS%,Team Win%
0,Steven Adams,C,26,OKC,63,63,26.7,4.5,7.6,0.592,...,9.3,2.3,0.8,1.1,1.5,1.9,10.9,20.5,0.604,0.611
1,Bam Adebayo,PF,22,MIA,72,72,33.6,6.1,11.0,0.557,...,10.2,5.1,1.1,1.3,2.8,2.5,15.9,20.3,0.598,0.603
2,LaMarcus Aldridge,C,34,SAS,53,53,33.1,7.4,15.0,0.493,...,7.4,2.4,0.7,1.6,1.4,2.4,18.9,19.7,0.571,0.458
3,Kyle Alexander,PF,23,MIA,2,0,6.5,0.5,1.0,0.500,...,1.5,0.0,0.0,0.0,0.5,0.5,1.0,4.7,0.500,0.603
4,Nickeil Alexander-Walker,SG,21,NOP,47,1,12.6,2.1,5.7,0.368,...,1.8,1.9,0.4,0.2,1.1,1.2,5.7,8.9,0.473,0.417
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
646,Trae Young,PG,21,ATL,60,60,35.3,9.1,20.8,0.437,...,4.3,9.3,1.1,0.1,4.8,1.7,29.6,23.9,0.595,0.299
647,Cody Zeller,C,27,CHO,58,39,23.1,4.3,8.3,0.524,...,7.1,1.5,0.7,0.4,1.3,2.4,11.1,18.8,0.576,0.354
648,Tyler Zeller,C,30,SAS,2,0,2.0,0.5,2.0,0.250,...,2.0,0.0,0.0,0.0,0.0,0.0,1.0,22.4,0.250,0.458
649,Ante Žižić,C,23,CLE,22,0,10.0,1.9,3.3,0.569,...,3.0,0.3,0.3,0.2,0.5,1.2,4.4,16.4,0.597,0.292


In [719]:
all_stars_2020 = ["LeBron James", "Anthony Davis", "Kawhi Leonard", "Luka Doncic", "James Harden", "Damian Lillard", 
             "Ben Simmons", "Nikola Jokic", "Jayson Tatum", "Chris Paul", "Russell Westbrook", "Domantas Sabonis", 
            "Giannis Antetokounmpo", "Joel Embiid", "Pascal Siakam", "Kemba Walker", "Trae Young", "Khris Middleton", 
            "Bam Adebayo", "Rudy Gobert", "Jimmy Butler", "Kyle Lowry", "Brandon Ingram", "Donovan Mitchell"]
is_allstar_2020 = []
for player in df2['Player']:
    if player in all_stars_2020:
        is_allstar_2020.append(1)
    else:
        is_allstar_2020.append(0)

print("# of all stars 2020 = ", len(all_stars_2020))
df2["all star"] = is_allstar_2020
df2 = df2.drop(['Tm', 'Pos'], axis=1)
df2

# of all stars 2020 =  24


,Player,Age,G,GS,MP,FG,FGA,FG%,3P,3PA,...,AST,STL,BLK,TOV,PF,PTS,PER,TS%,Team Win%,all star
0,Steven Adams,26,63,63,26.7,4.5,7.6,0.592,0.0,0.0,...,2.3,0.8,1.1,1.5,1.9,10.9,20.5,0.604,0.611,0
1,Bam Adebayo,22,72,72,33.6,6.1,11.0,0.557,0.0,0.2,...,5.1,1.1,1.3,2.8,2.5,15.9,20.3,0.598,0.603,1
2,LaMarcus Aldridge,34,53,53,33.1,7.4,15.0,0.493,1.2,3.0,...,2.4,0.7,1.6,1.4,2.4,18.9,19.7,0.571,0.458,0
3,Kyle Alexander,23,2,0,6.5,0.5,1.0,0.500,0.0,0.0,...,0.0,0.0,0.0,0.5,0.5,1.0,4.7,0.500,0.603,0
4,Nickeil Alexander-Walker,21,47,1,12.6,2.1,5.7,0.368,1.0,2.8,...,1.9,0.4,0.2,1.1,1.2,5.7,8.9,0.473,0.417,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
646,Trae Young,21,60,60,35.3,9.1,20.8,0.437,3.4,9.5,...,9.3,1.1,0.1,4.8,1.7,29.6,23.9,0.595,0.299,1
647,Cody Zeller,27,58,39,23.1,4.3,8.3,0.524,0.3,1.3,...,1.5,0.7,0.4,1.3,2.4,11.1,18.8,0.576,0.354,0
648,Tyler Zeller,30,2,0,2.0,0.5,2.0,0.250,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,22.4,0.250,0.458,0
649,Ante Žižić,23,22,0,10.0,1.9,3.3,0.569,0.0,0.0,...,0.3,0.3,0.2,0.5,1.2,4.4,16.4,0.597,0.292,0


In [720]:
i = 0
NaN_i = []
for t in df2['3P%']:
    if math.isnan(t):
        NaN_i.append(i)
    i+=1

for i in NaN_i:
    df2['3P%'].values[i] = 0.0

df2['3P%']

0      0.333
1      0.143
2      0.389
3      0.000
4      0.346
       ...  
646    0.361
647    0.240
648    0.000
649    0.000
650    0.000
Name: 3P%, Length: 591, dtype: float64

In [721]:
counter = 0
for d in df2:
    if counter != 0:
        for i in range(len(df2[d])):
            cell_value = df2[d].values[i]
            if math.isnan(cell_value):
                df2[d].values[i] = 0.0
    counter = 1

#df2 = df2[0:4]
df2

,Player,Age,G,GS,MP,FG,FGA,FG%,3P,3PA,...,AST,STL,BLK,TOV,PF,PTS,PER,TS%,Team Win%,all star
0,Steven Adams,26,63,63,26.7,4.5,7.6,0.592,0.0,0.0,...,2.3,0.8,1.1,1.5,1.9,10.9,20.5,0.604,0.611,0
1,Bam Adebayo,22,72,72,33.6,6.1,11.0,0.557,0.0,0.2,...,5.1,1.1,1.3,2.8,2.5,15.9,20.3,0.598,0.603,1
2,LaMarcus Aldridge,34,53,53,33.1,7.4,15.0,0.493,1.2,3.0,...,2.4,0.7,1.6,1.4,2.4,18.9,19.7,0.571,0.458,0
3,Kyle Alexander,23,2,0,6.5,0.5,1.0,0.500,0.0,0.0,...,0.0,0.0,0.0,0.5,0.5,1.0,4.7,0.500,0.603,0
4,Nickeil Alexander-Walker,21,47,1,12.6,2.1,5.7,0.368,1.0,2.8,...,1.9,0.4,0.2,1.1,1.2,5.7,8.9,0.473,0.417,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
646,Trae Young,21,60,60,35.3,9.1,20.8,0.437,3.4,9.5,...,9.3,1.1,0.1,4.8,1.7,29.6,23.9,0.595,0.299,1
647,Cody Zeller,27,58,39,23.1,4.3,8.3,0.524,0.3,1.3,...,1.5,0.7,0.4,1.3,2.4,11.1,18.8,0.576,0.354,0
648,Tyler Zeller,30,2,0,2.0,0.5,2.0,0.250,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,22.4,0.250,0.458,0
649,Ante Žižić,23,22,0,10.0,1.9,3.3,0.569,0.0,0.0,...,0.3,0.3,0.2,0.5,1.2,4.4,16.4,0.597,0.292,0


In [722]:
validation_vec = []
validation_lab = []

for x, row in df2.iterrows():
    v_vec = []
    label = int(row[28])
    validation_lab.append(label)
    for i in range(len(row)):
        data = row[i]
        if i != len(row)-1 and i != 0:
            v_vec.append(data)
    validation_vec.append(v_vec)

validation_vec = torch.tensor(validation_vec)
validation_lab = torch.tensor(validation_lab)
print("validation_vec[1] = ", validation_vec[1])
print("validation_lab[1] = ", validation_lab[1])
print("size^ = ", len(validation_vec[1]))
print("size validation_vec = ", len(validation_vec))

validation_vec[1] =  tensor([22.0000, 72.0000, 72.0000, 33.6000,  6.1000, 11.0000,  0.5570,  0.0000,
         0.2000,  0.1430,  6.1000, 10.8000,  0.5640,  0.5580,  3.7000,  5.3000,
         0.6910, 10.2000,  5.1000,  1.1000,  1.3000,  2.8000,  2.5000, 15.9000,
        20.3000,  0.5980,  0.6030])
validation_lab[1] =  tensor(1)
size^ =  27
size validation_vec =  591


In [723]:
index1 = 0
for i, row in df1.iterrows():
    print("index = ", index1)
    print(row)
    print(" ")
    index1+=1

index =  0
Player       Precious Achiuwa
Age                        21
G                          28
GS                          2
MP                       14.6
FG                        2.6
FGA                       4.4
FG%                      0.59
3P                        0.0
3PA                       0.0
3P%                       0.0
2P                        2.6
2PA                       4.4
2P%                      0.59
eFG%                     0.59
FT                        1.3
FTA                       2.4
FT%                     0.561
TRB                       4.0
AST                       0.6
STL                       0.4
BLK                       0.5
TOV                       1.0
PF                        1.9
PTS                       6.5
PER                      15.1
TS%                     0.599
Team Win%               0.556
all star                    0
Name: 0, dtype: object
 
index =  1
Player       Jaylen Adams
Age                    24
G                       6
GS   

Name: 139, dtype: object
 
index =  139
Player       Yogi Ferrell
Age                    27
G                       2
GS                      0
MP                   20.0
FG                    4.0
FGA                  10.5
FG%                 0.381
3P                    1.5
3PA                   4.5
3P%                 0.333
2P                    2.5
2PA                   6.0
2P%                 0.417
eFG%                0.452
FT                    0.0
FTA                   1.0
FT%                   0.0
TRB                   3.5
AST                   2.5
STL                   1.5
BLK                   0.5
TOV                   1.5
PF                    2.0
PTS                   9.5
PER                  12.7
TS%                 0.434
Team Win%           0.306
all star                0
Name: 140, dtype: object
 
index =  140
Player       Dorian Finney-Smith
Age                           27
G                             19
GS                            19
MP                          30.7
F

Name: 149, dtype: object
 
index =  149
Player       Daniel Gafford
Age                      22
G                        23
GS                       11
MP                     14.3
FG                      2.1
FGA                     3.2
FG%                   0.671
3P                      0.0
3PA                     0.0
3P%                     0.0
2P                      2.1
2PA                     3.2
2P%                   0.671
eFG%                  0.671
FT                      1.1
FTA                     1.6
FT%                   0.676
TRB                     3.3
AST                     0.7
STL                     0.5
BLK                     1.3
TOV                     0.8
PF                      2.0
PTS                     5.3
PER                    17.6
TS%                   0.689
Team Win%             0.431
all star                  0
Name: 150, dtype: object
 
index =  150
Player       Danilo Gallinari
Age                        32
G                          16
GS                

index =  240
Player       Tyus Jones
Age                  24
G                    24
GS                    8
MP                 23.5
FG                  3.3
FGA                 8.0
FG%               0.406
3P                  0.7
3PA                 2.8
3P%                0.25
2P                  2.5
2PA                 5.2
2P%               0.492
eFG%              0.451
FT                  0.5
FTA                 0.6
FT%                 0.8
TRB                 2.8
AST                 5.3
STL                 1.3
BLK                 0.1
TOV                 1.0
PF                  0.5
PTS                 7.7
PER                14.2
TS%               0.466
Team Win%         0.528
all star              0
Name: 242, dtype: object
 
index =  241
Player       DeAndre Jordan
Age                      32
G                        28
GS                       22
MP                     21.6
FG                      3.2
FGA                     4.0
FG%                   0.788
3P                      0.0

index =  362
Player       Elfrid Payton
Age                     26
G                       30
GS                      30
MP                    27.7
FG                     5.2
FGA                   11.8
FG%                  0.441
3P                     0.5
3PA                    1.9
3P%                   0.25
2P                     4.7
2PA                    9.9
2P%                  0.477
eFG%                  0.46
FT                     1.4
FTA                    1.9
FT%                  0.719
TRB                    3.8
AST                    3.6
STL                    0.6
BLK                    0.2
TOV                    1.9
PF                     2.0
PTS                   12.2
PER                   11.9
TS%                  0.484
Team Win%            0.569
all star                 0
Name: 367, dtype: object
 
index =  363
Player       Norvel Pelle
Age                    27
G                       3
GS                      0
MP                    9.3
FG                    1.0
FGA     

Name: 470, dtype: object
 
index =  463
Player       Yuta Watanabe
Age                     26
G                       18
GS                       0
MP                    12.4
FG                     1.0
FGA                    2.8
FG%                  0.353
3P                     0.7
3PA                    1.6
3P%                  0.429
2P                     0.3
2PA                    1.3
2P%                  0.261
eFG%                 0.471
FT                     0.4
FTA                    0.4
FT%                    1.0
TRB                    3.3
AST                    0.4
STL                    0.5
BLK                    0.6
TOV                    0.4
PF                     1.1
PTS                    3.1
PER                   11.1
TS%                  0.514
Team Win%            0.375
all star                 0
Name: 471, dtype: object
 
index =  464
Player       Tremont Waters
Age                      23
G                         9
GS                        1
MP                     10

In [724]:
index2 = 0
for i, row in df2.iterrows():
    print("index = ", index2)
    print(row)
    print(" ")
    index2+=1

index =  0
Player       Steven Adams
Age                    26
G                      63
GS                     63
MP                   26.7
FG                    4.5
FGA                   7.6
FG%                 0.592
3P                    0.0
3PA                   0.0
3P%                 0.333
2P                    4.5
2PA                   7.5
2P%                 0.594
eFG%                0.593
FT                    1.9
FTA                   3.2
FT%                 0.582
TRB                   9.3
AST                   2.3
STL                   0.8
BLK                   1.1
TOV                   1.5
PF                    1.9
PTS                  10.9
PER                  20.5
TS%                 0.604
Team Win%           0.611
all star                0
Name: 0, dtype: object
 
index =  1
Player       Bam Adebayo
Age                   22
G                     72
GS                    72
MP                  33.6
FG                   6.1
FGA                 11.0
FG%                0.557

index =  20
Player       D.J. Augustin
Age                     32
G                       57
GS                      13
MP                    24.9
FG                     3.2
FGA                    8.1
FG%                  0.399
3P                     1.2
3PA                    3.5
3P%                  0.348
2P                     2.0
2PA                    4.6
2P%                  0.438
eFG%                 0.475
FT                     2.8
FTA                    3.2
FT%                   0.89
TRB                    2.1
AST                    4.6
STL                    0.6
BLK                    0.0
TOV                    1.5
PF                     1.3
PTS                   10.5
PER                   14.0
TS%                  0.554
Team Win%            0.292
all star                 0
Name: 21, dtype: object
 
index =  21
Player       Deandre Ayton
Age                     21
G                       38
GS                      32
MP                    32.5
FG                     8.2
FGA  

Player       Bogdan Bogdanović
Age                         27
G                           61
GS                          28
MP                        29.0
FG                         5.5
FGA                       12.6
FG%                       0.44
3P                         2.7
3PA                        7.2
3P%                      0.372
2P                         2.8
2PA                        5.3
2P%                      0.532
eFG%                     0.547
FT                         1.4
FTA                        1.9
FT%                      0.741
TRB                        3.4
AST                        3.4
STL                        1.0
BLK                        0.2
TOV                        1.7
PF                         2.2
PTS                       15.1
PER                       15.1
TS%                      0.565
Team Win%                0.431
all star                     0
Name: 59, dtype: object
 
index =  54
Player       Bojan Bogdanović
Age                        30
G  

Name: 147, dtype: object
 
index =  131
Player       Allen Crabbe
Age                    27
G                      28
GS                      1
MP                   18.6
FG                    1.9
FGA                   5.1
FG%                 0.364
3P                    1.1
3PA                   3.3
3P%                 0.323
2P                    0.8
2PA                   1.8
2P%                  0.44
eFG%                0.469
FT                    0.3
FTA                   0.4
FT%                  0.75
TRB                   2.3
AST                   1.0
STL                   0.5
BLK                   0.1
TOV                   0.4
PF                    1.3
PTS                   5.1
PER                   7.2
TS%                 0.482
Team Win%           0.299
all star                0
Name: 149, dtype: object
 
index =  132
Player       Allen Crabbe
Age                    27
G                       9
GS                      0
MP                   14.6
FG                    1.2
FGA       

Name: 215, dtype: object
 
index =  190
Player       Wenyen Gabriel
Age                      22
G                        11
GS                        0
MP                      5.5
FG                      0.5
FGA                     1.5
FG%                   0.353
3P                      0.1
3PA                     0.7
3P%                   0.125
2P                      0.5
2PA                     0.8
2P%                   0.556
eFG%                  0.382
FT                      0.5
FTA                     0.9
FT%                     0.6
TRB                     0.9
AST                     0.3
STL                     0.3
BLK                     0.2
TOV                     0.5
PF                      0.6
PTS                     1.7
PER                     7.4
TS%                   0.444
Team Win%             0.431
all star                  0
Name: 217, dtype: object
 
index =  191
Player       Wenyen Gabriel
Age                      22
G                        19
GS                      

...

index =  373
Player       Jordan McRae
Age                    28
G                       4
GS                      0
MP                   24.5
FG                    3.5
FGA                  10.8
FG%                 0.326
3P                    0.8
3PA                   4.0
3P%                 0.188
2P                    2.8
2PA                   6.8
2P%                 0.407
eFG%                 0.36
FT                    4.0
FTA                   5.5
FT%                 0.727
TRB                   3.8
AST                   1.8
STL                   0.0
BLK                   0.0
TOV                   1.3
PF                    2.5
PTS                  11.8
PER                   6.5
TS%                 0.446
Team Win%           0.303
all star                0
Name: 414, dtype: object
 
index =  374
Player       Nicolò Melli
Age                    29
G                      60
GS                      8
MP                   17.4
FG                    2.3
FGA                   5.4
FG%        

 
index =  410
Player       Malik Newman
Age                    22
G                       1
GS                      0
MP                    4.0
FG                    0.0
FGA                   2.0
FG%                   0.0
3P                    0.0
3PA                   1.0
3P%                   0.0
2P                    0.0
2PA                   1.0
2P%                   0.0
eFG%                  0.0
FT                    2.0
FTA                   2.0
FT%                   1.0
TRB                   0.0
AST                   0.0
STL                   0.0
BLK                   0.0
TOV                   1.0
PF                    0.0
PTS                   2.0
PER                 -11.4
TS%                 0.347
Team Win%           0.292
all star                0
Name: 454, dtype: object
 
index =  411
Player       Georges Niang
Age                     26
G                       66
GS                       1
MP                    14.0
FG                     2.1
FGA                    4.9
FG

 
index =  498
Player       Chris Silva
Age                   23
G                     44
GS                     0
MP                   7.9
FG                   1.1
FGA                  1.8
FG%                0.615
3P                   0.0
3PA                  0.1
3P%                  0.0
2P                   1.1
2PA                  1.7
2P%                 0.64
eFG%               0.615
FT                   0.8
FTA                  1.3
FT%                0.673
TRB                  2.9
AST                  0.5
STL                  0.2
BLK                  0.5
TOV                  0.7
PF                   1.9
PTS                  3.0
PER                 17.3
TS%                0.651
Team Win%          0.603
all star               0
Name: 547, dtype: object
 
index =  499
Player       Ben Simmons
Age                   23
G                     57
GS                    57
MP                  35.4
FG                   6.6
FGA                 11.4
FG%                 0.58
3P                  

Name: 581, dtype: object
 
index =  530
Player       Gary Trent Jr.
Age                      21
G                        61
GS                        8
MP                     21.8
FG                      3.2
FGA                     7.2
FG%                   0.444
3P                      1.8
3PA                     4.4
3P%                   0.418
2P                      1.4
2PA                     2.8
2P%                   0.486
eFG%                  0.571
FT                      0.6
FTA                     0.7
FT%                   0.822
TRB                     1.6
AST                     1.0
STL                     0.8
BLK                     0.3
TOV                     0.3
PF                      1.5
PTS                     8.9
PER                    12.9
TS%                   0.587
Team Win%             0.583
all star                  0
Name: 582, dtype: object
 
index =  531
Player       Allonzo Trier
Age                     24
G                       24
GS                       1


Name: 645, dtype: object
 
index =  586
Player       Trae Young
Age                  21
G                    60
GS                   60
MP                 35.3
FG                  9.1
FGA                20.8
FG%               0.437
3P                  3.4
3PA                 9.5
3P%               0.361
2P                  5.7
2PA                11.4
2P%               0.501
eFG%              0.519
FT                  8.0
FTA                 9.3
FT%                0.86
TRB                 4.3
AST                 9.3
STL                 1.1
BLK                 0.1
TOV                 4.8
PF                  1.7
PTS                29.6
PER                23.9
TS%               0.595
Team Win%         0.299
all star              1
Name: 646, dtype: object
 
index =  587
Player       Cody Zeller
Age                   27
G                     58
GS                    39
MP                  23.1
FG                   4.3
FGA                  8.3
FG%                0.524
3P                   0.3

In [725]:
def fe0(X):
    return_x = []
    for x in X:
        return_x.append(x[23])
    return torch.tensor(return_x)

def fe1(X):
    return X;

fe = [fe0,fe1]

In [726]:
class GNB:
    '''
    Constructor
    input: uni - set as True for univariate model
           ID_FE - set as 2 for fe2, otherwise don't change
    '''
    def __init__ (self, uni=True, ID_FE=0):
        self.uni = uni
        # do not change these
        self.prior = None       # list of the priors. Made from empirical counts
        self.dists = None       # list of Gaussian distributions likelihood. size = number of classes
        self.num_c = 0          # number of classes
        self.ID_FE = max(0 if self.uni else 1, ID_FE) # select feature extractor
    
    '''
    Train your model
    input: X - training data
           y - training labels
           uni - set as True for univariate model
    '''
    def train(self, X, y):
        self.dists = list()
        uniq_y = torch.unique(y)                      # get all labels
        self.prior = torch.zeros(uniq_y.shape[0])     # initialize prior P(Y) to zeros
        self.num_c = uniq_y.shape[0]                  # number of classes  
        
        print("self.dists = ", self.dists)
        print("uniq_y = ", uniq_y)
        print("self.prior = ", self.prior)
        print("self.num_c = ", self.num_c)
        print(" ")
        
        for idx in range(self.num_c):           # for each label
            print("counter = ", idx)
            mask = torch.eq(y,idx)
            x_by_label = X[mask]           # select all observations with label
            num_of_x = x_by_label.size()[0]
            
            self.prior[idx] = num_of_x/y.size()[0]            ##### TODO : Estimate your model's prior P(Y) (parts a, b, and c)
            # raise NotImplementedError           ##### TODO : Remove this line before running
            print("self.prior[" + str(idx) + "] = " + str(self.prior[idx]))
            print("mask = ", mask)
            print("x_by_label = ", x_by_label)
            print("num_of_x = ", num_of_x)
            print("x_by_label.size() = ", x_by_label.size())
            x = fe[self.ID_FE](x_by_label)      # extract the feature vector
            print("x feature = ", x)
            if self.uni: ##### Univariate gaussian (parts a and b)

                ########### TODO: Estimate the univariate Gaussian conditional distributions (parts a and b)           
                ##### Hint: Compute the empirical mean and variance. Then you can call PyTorch's "Normal" 
                #####       function with the correct arguments to build the distribution. We have already 
                #####       imported the relevant function for you. You can refer to PyTorch's official       
                #####       documentation for more info. 
                x_mean = x.mean()                  ##### TODO : calculate the mean value 
                print("x feature mean = ", x_mean)
                x_var = x.var()                  ##### TODO : calculate the variance
                print("x feature variance = ", x_var)
                gaussian_dist = normal.Normal(x_mean, torch.sqrt(x_var))          ##### TODO : replace 'None' with distribution's constructor
                print("gaussian dist = ", gaussian_dist)
                # raise NotImplementedError     ##### TODO : remove this line before running

            else: ##### Multivariate Gaussian. (part c)

                ########### TODO: Estimate the multivariate Gaussian conditional distributions (part c)           
                ##### Hint: In this part, you will call PyTorch's "MultivariateNormal" function to build    
                #####       a multivariate normal distribution. The function is already imported.     
                xT = x.T
                print("xT = ", xT)
                n_sample = xT.shape[0]
                print("num samples = ", n_sample)
                constant = 0.0000001
                x_mean = torch.mean(xT, dim=1)             ##### TODO : estimate the mean vector
                print("xT mean = ", x_mean)
                print("xT mean size = ", x_mean.size())
                x_minus_mu = x - x_mean
                print("x - mu = ", x_minus_mu)
                print("x - mu size = ", x_minus_mu.size())
                x_cov = torch.matmul(x_minus_mu.T,  x_minus_mu) / n_sample                  ##### TODO : estimate the covariance matrix
                id_matrix = torch.tensor([constant]*27)
                small_constant_id_matrix = torch.diag(id_matrix)
                x_cov = x_cov + small_constant_id_matrix
                
                gaussian_dist = multivariate_normal.MultivariateNormal(x_mean, x_cov)       ##### TODO : replace 'None' with distribution's constructor
                # raise NotImplementedError     ##### TODO : remove this line before running
                print("gaussian_dist = ", gaussian_dist)
            print("----------------------------------------------------")
            
            self.dists.append(gaussian_dist)
        return

    '''
        Use Bayes rule to predict on one sample
    '''
    def predict(self, x):
        #print("\t inside predict function")
        print("\t x = ", x)
        x = fe[self.ID_FE](torch.reshape(x,(1,-1)))
        print("\t x reshape = ", x)
        result = []
        #print("\t self.num_c = ", self.num_c)
        for i in range(self.num_c):
            term1 = self.dists[i].log_prob(x)
            term2 = torch.log(self.prior[i])
            #print("\t \t term1, term2 = " + str(term1) + ", " + str(term2))
            result.append(term1+term2)
        #result = [self.dists[i].log_prob(x) + torch.log(self.prior[i]) for i in range(self.num_c)]
        #print("\t result = ", result)
        result_tensor = torch.tensor(result)
        #print("\t result tensor = ", result_tensor)
        to_return = torch.argmax(result_tensor)
        #print("\t return = ", to_return)
        return to_return

    '''
        Evaluate classification accuracy
    '''
    def evaluate(self, X, y):
        correct = 0
        wrong = 0
        #print("X = ", X)
        #print("y = ", y)
        for i in range(X.shape[0]):
            #print("index = " + str(i))
            prediction = self.predict(X[i])
            actual = y[i]
            #print("\t prediction = ", prediction)
            #print("\t actual = ", actual)
            if prediction == actual:
                correct = correct + 1
            else:
#                 print("\t \t ^incorrect prediction at index = ", i)
                print("index = ", i)
                print("\t prediction, actual = " + str(prediction.item()) + ", " + str(actual.item()))
                wrong += 1
        print('The classification accuracy is {:.3f}'.format(correct/X.shape[0]))
        print("wrong = ", wrong)
        return

In [727]:
model_uni = GNB(uni=True)
model_uni.train(train_vec, train_lab)

self.dists =  []
uniq_y =  tensor([0, 1])
self.prior =  tensor([0., 0.])
self.num_c =  2
 
counter =  0
self.prior[0] = tensor(0.9427)
mask =  tensor([ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True, False,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True, False,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True, False,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True, False,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True, False,  True,
         True,  True,  True,  True,  True,  True,  True, False, False,  True,

x_by_label =  tensor([[26.0000, 27.0000, 27.0000, 33.6000, 10.3000, 18.4000,  0.5630,  1.1000,
          4.0000,  0.2900,  9.2000, 14.4000,  0.6380,  0.5940,  6.4000,  9.9000,
          0.6420, 11.4000,  5.8000,  1.3000,  1.3000,  3.7000,  3.1000, 28.2000,
         28.3000,  0.6200,  0.6390],
        [27.0000, 24.0000, 24.0000, 35.3000, 11.4000, 24.0000,  0.4730,  2.5000,
          7.4000,  0.3370,  8.9000, 16.6000,  0.5340,  0.5250,  7.5000,  8.4000,
          0.8960,  5.2000,  4.7000,  1.3000,  0.5000,  3.1000,  2.5000, 32.8000,
         24.8000,  0.5910,  0.4270],
        [24.0000, 23.0000, 23.0000, 35.2000,  9.0000, 18.2000,  0.4920,  2.0000,
          5.8000,  0.3530,  6.9000, 12.4000,  0.5560,  0.5480,  4.4000,  5.3000,
          0.8290,  3.7000,  4.4000,  0.9000,  0.3000,  3.5000,  3.3000, 24.4000,
         17.2000,  0.5930,  0.7080],
        [24.0000, 26.0000, 26.0000, 33.4000,  9.9000, 19.5000,  0.5060,  2.4000,
          5.9000,  0.4090,  7.5000, 13.6000,  0.5480,  0.5680,  3

In [728]:
model_uni.evaluate(train_vec, train_lab)

	 x =  tensor([21.0000, 28.0000,  2.0000, 14.6000,  2.6000,  4.4000,  0.5900,  0.0000,
         0.0000,  0.0000,  2.6000,  4.4000,  0.5900,  0.5900,  1.3000,  2.4000,
         0.5610,  4.0000,  0.6000,  0.4000,  0.5000,  1.0000,  1.9000,  6.5000,
        15.1000,  0.5990,  0.5560])
	 x reshape =  tensor([6.5000])
	 x =  tensor([24.0000,  6.0000,  0.0000,  2.8000,  0.2000,  1.3000,  0.1250,  0.0000,
         0.3000,  0.0000,  0.2000,  1.0000,  0.1670,  0.1250,  0.0000,  0.0000,
         0.0000,  0.5000,  0.3000,  0.0000,  0.0000,  0.0000,  0.2000,  0.3000,
        -6.9000,  0.1250,  0.6390])
	 x reshape =  tensor([0.3000])
	 x =  tensor([27.0000, 27.0000, 27.0000, 28.1000,  3.5000,  5.8000,  0.6030,  0.0000,
         0.0000,  0.0000,  3.5000,  5.7000,  0.6060,  0.6030,  1.1000,  2.3000,
         0.4680,  8.9000,  2.1000,  1.0000,  0.6000,  1.7000,  1.9000,  8.0000,
        15.9000,  0.5920,  0.4310])
	 x reshape =  tensor([8.])
	 x =  tensor([23.0000, 26.0000, 26.0000, 33.6000,  7.4000,

	 x =  tensor([22.0000,  1.0000,  0.0000,  5.0000,  1.0000,  2.0000,  0.5000,  0.0000,
         0.0000,  0.0000,  1.0000,  2.0000,  0.5000,  0.5000,  1.0000,  1.0000,
         1.0000,  1.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  3.0000,
        17.8000,  0.6150,  0.4270])
	 x reshape =  tensor([3.])
	 x =  tensor([30.0000, 10.0000,  1.0000, 21.1000,  3.1000,  6.6000,  0.4700,  1.6000,
         3.8000,  0.4210,  1.5000,  2.8000,  0.5360,  0.5910,  0.7000,  0.9000,
         0.7780,  1.8000,  1.4000,  0.7000,  0.1000,  0.9000,  2.6000,  8.5000,
        10.4000,  0.6070,  0.5560])
	 x reshape =  tensor([8.5000])
	 x =  tensor([23.0000, 10.0000,  1.0000, 11.7000,  1.9000,  3.6000,  0.5280,  0.0000,
         0.1000,  0.0000,  1.9000,  3.5000,  0.5430,  0.5280,  0.3000,  0.5000,
         0.6000,  5.0000,  0.6000,  0.3000,  0.6000,  0.1000,  1.1000,  4.1000,
        20.3000,  0.5370,  0.6810])
	 x reshape =  tensor([4.1000])
	 x =  tensor([24.0000,  8.0000,  0.0000,  2.8000,  0.4000,

	 x =  tensor([25.0000, 23.0000,  0.0000, 11.1000,  1.1000,  3.2000,  0.3420,  0.6000,
         2.1000,  0.2860,  0.5000,  1.0000,  0.4580,  0.4380,  0.1000,  0.1000,
         1.0000,  1.4000,  1.1000,  0.4000,  0.3000,  0.2000,  1.2000,  2.9000,
         8.3000,  0.4470,  0.7080])
	 x reshape =  tensor([2.9000])
	 x =  tensor([21.0000, 16.0000, 16.0000, 26.6000,  4.8000,  8.6000,  0.5510,  0.3000,
         0.9000,  0.2670,  4.5000,  7.7000,  0.5850,  0.5650,  3.3000,  4.5000,
         0.7220,  7.8000,  2.2000,  0.7000,  0.7000,  1.9000,  3.0000, 13.0000,
        17.1000,  0.6130,  0.4310])
	 x reshape =  tensor([13.])
	 x =  tensor([29.0000, 10.0000,  5.0000, 24.6000,  2.9000,  8.2000,  0.3540,  0.2000,
         1.7000,  0.1180,  2.7000,  6.5000,  0.4150,  0.3660,  1.7000,  2.3000,
         0.7390,  4.7000,  3.0000,  0.5000,  0.4000,  1.6000,  2.7000,  7.7000,
         8.2000,  0.4180,  0.2920])
	 x reshape =  tensor([7.7000])
	 x =  tensor([26.0000, 22.0000,  0.0000, 18.7000,  2.0000

	 x =  tensor([22.0000, 13.0000,  1.0000, 10.3000,  1.8000,  4.2000,  0.4360,  0.8000,
         2.3000,  0.3330,  1.1000,  1.9000,  0.5600,  0.5270,  0.5000,  0.7000,
         0.7780,  1.0000,  0.4000,  0.3000,  0.1000,  0.2000,  1.2000,  5.0000,
        12.2000,  0.5510,  0.5000])
	 x reshape =  tensor([5.])
	 x =  tensor([20.0000, 12.0000,  0.0000,  8.0000,  1.2000,  2.4000,  0.4830,  0.4000,
         1.5000,  0.2780,  0.8000,  0.9000,  0.8180,  0.5690,  0.5000,  0.8000,
         0.6670,  1.0000,  0.3000,  0.3000,  0.3000,  0.3000,  1.3000,  3.3000,
        12.2000,  0.5920,  0.5830])
	 x reshape =  tensor([3.3000])
	 x =  tensor([33.0000, 21.0000, 18.0000, 22.9000,  3.7000,  8.0000,  0.4560,  2.7000,
         6.2000,  0.4380,  1.0000,  1.9000,  0.5130,  0.6240,  0.6000,  0.8000,
         0.7500,  2.0000,  1.1000,  0.4000,  0.2000,  0.7000,  1.5000, 10.6000,
        12.8000,  0.6330,  0.2780])
	 x reshape =  tensor([10.6000])
	 x =  tensor([26.0000, 23.0000, 23.0000, 32.5000,  9.4000

	 x reshape =  tensor([3.1000])
	 x =  tensor([22.0000, 22.0000, 22.0000, 33.7000,  7.9000, 15.6000,  0.5030,  1.9000,
         4.9000,  0.3830,  6.0000, 10.8000,  0.5570,  0.5630,  5.0000,  6.3000,
         0.7840,  5.4000,  6.5000,  0.8000,  0.6000,  2.9000,  2.0000, 22.5000,
        21.6000,  0.6120,  0.3060])
	 x reshape =  tensor([22.5000])
index =  158
	 prediction, actual = 1, 0
	 x =  tensor([28.0000, 10.0000,  1.0000,  6.5000,  0.3000,  1.0000,  0.3000,  0.1000,
         0.7000,  0.1430,  0.2000,  0.3000,  0.6670,  0.3500,  0.3000,  0.4000,
         0.7500,  1.9000,  0.3000,  0.3000,  0.2000,  0.3000,  0.3000,  1.0000,
         7.9000,  0.4250,  0.4270])
	 x reshape =  tensor([1.])
	 x =  tensor([28.0000, 29.0000, 29.0000, 30.3000,  5.4000,  8.5000,  0.6400,  0.0000,
         0.0000,  0.0000,  5.4000,  8.5000,  0.6420,  0.6400,  3.3000,  5.8000,
         0.5810, 13.4000,  1.3000,  0.4000,  2.7000,  1.6000,  2.2000, 14.2000,
        23.2000,  0.6440,  0.7220])
	 x reshape =  te

...

	 x =  tensor([32.0000,  2.0000,  2.0000, 23.0000,  3.0000,  9.0000,  0.3330,  1.5000,
         4.5000,  0.3330,  1.5000,  4.5000,  0.3330,  0.4170,  2.0000,  2.5000,
         0.8000,  6.0000,  2.5000,  0.5000,  0.0000,  1.5000,  0.5000,  9.5000,
        11.2000,  0.4700,  0.3060])
	 x reshape =  tensor([9.5000])
	 x =  tensor([34.0000, 25.0000, 25.0000, 34.4000,  6.0000, 13.2000,  0.4520,  2.8000,
         7.1000,  0.3880,  3.2000,  6.1000,  0.5260,  0.5560,  3.0000,  3.5000,
         0.8740,  5.4000,  6.4000,  1.2000,  0.2000,  2.9000,  3.2000, 17.7000,
        16.3000,  0.6010,  0.3750])
	 x reshape =  tensor([17.7000])
	 x =  tensor([25.0000, 30.0000,  5.0000, 19.7000,  2.6000,  6.4000,  0.4040,  1.5000,
         4.1000,  0.3660,  1.1000,  2.3000,  0.4710,  0.5210,  0.6000,  0.8000,
         0.7500,  2.4000,  1.3000,  0.6000,  0.2000,  0.7000,  1.6000,  7.3000,
         9.8000,  0.5380,  0.6670])
	 x reshape =  tensor([7.3000])
	 x =  tensor([25.0000, 14.0000,  2.0000, 11.8000,  1.

	 x =  tensor([27.0000, 29.0000,  0.0000, 13.1000,  1.8000,  4.7000,  0.3900,  1.0000,
         3.2000,  0.3230,  0.8000,  1.5000,  0.5350,  0.5000,  0.2000,  0.2000,
         1.0000,  2.2000,  0.8000,  0.4000,  0.1000,  0.7000,  1.5000,  4.9000,
         8.3000,  0.5100,  0.7220])
	 x reshape =  tensor([4.9000])
	 x =  tensor([20.0000, 15.0000,  0.0000,  7.5000,  1.1000,  2.1000,  0.5000,  0.7000,
         1.3000,  0.5000,  0.4000,  0.8000,  0.5000,  0.6560,  0.1000,  0.3000,
         0.5000,  0.9000,  0.0000,  0.1000,  0.1000,  0.0000,  0.5000,  2.9000,
        12.5000,  0.6520,  0.6530])
	 x reshape =  tensor([2.9000])
	 x =  tensor([26.0000, 25.0000,  3.0000, 19.0000,  1.8000,  3.1000,  0.5640,  0.0000,
         0.1000,  0.0000,  1.8000,  3.0000,  0.5790,  0.5640,  0.5000,  0.9000,
         0.5450,  5.3000,  0.5000,  0.9000,  1.9000,  0.9000,  2.2000,  4.0000,
        14.3000,  0.5700,  0.5690])
	 x reshape =  tensor([4.])
	 x =  tensor([21.0000, 15.0000,  0.0000, 15.7000,  3.5000,

	 x reshape =  tensor([6.6000])
	 x =  tensor([26.0000, 30.0000, 30.0000, 27.7000,  5.2000, 11.8000,  0.4410,  0.5000,
         1.9000,  0.2500,  4.7000,  9.9000,  0.4770,  0.4600,  1.4000,  1.9000,
         0.7190,  3.8000,  3.6000,  0.6000,  0.2000,  1.9000,  2.0000, 12.2000,
        11.9000,  0.4840,  0.5690])
	 x reshape =  tensor([12.2000])
	 x =  tensor([27.0000,  3.0000,  0.0000,  9.3000,  1.0000,  2.3000,  0.4290,  0.0000,
         0.0000,  0.0000,  1.0000,  2.3000,  0.4290,  0.4290,  0.0000,  0.0000,
         0.0000,  2.3000,  0.0000,  0.0000,  1.0000,  1.0000,  3.0000,  2.0000,
         2.1000,  0.4290,  0.6670])
	 x reshape =  tensor([2.])
	 x =  tensor([20.0000, 14.0000,  0.0000, 10.5000,  1.5000,  3.6000,  0.4120,  0.1000,
         0.9000,  0.1670,  1.4000,  2.8000,  0.4870,  0.4310,  0.4000,  0.6000,
         0.6670,  3.6000,  0.9000,  0.1000,  0.4000,  0.8000,  1.9000,  3.6000,
         9.7000,  0.4550,  0.6670])
	 x reshape =  tensor([3.6000])
	 x =  tensor([25.0000,  9

	 x =  tensor([26.0000, 26.0000, 26.0000, 33.3000,  7.5000, 15.2000,  0.4900,  3.4000,
         7.7000,  0.4450,  4.0000,  7.5000,  0.5360,  0.6020,  2.2000,  2.8000,
         0.7950,  4.1000,  3.0000,  1.3000,  0.5000,  1.7000,  2.0000, 20.6000,
        18.3000,  0.6250,  0.4580])
	 x reshape =  tensor([20.6000])
	 x =  tensor([30.0000, 27.0000, 12.0000, 25.1000,  2.5000,  6.7000,  0.3700,  0.6000,
         2.1000,  0.2760,  1.9000,  4.6000,  0.4150,  0.4140,  1.6000,  2.0000,
         0.8300,  3.4000,  6.5000,  1.3000,  0.0000,  2.1000,  2.2000,  7.2000,
        11.2000,  0.4750,  0.3190])
	 x reshape =  tensor([7.2000])
	 x =  tensor([24.0000, 20.0000, 19.0000, 29.2000,  7.0000, 16.3000,  0.4260,  3.0000,
         7.4000,  0.3990,  4.0000,  8.9000,  0.4490,  0.5170,  2.4000,  3.2000,
         0.7500,  2.6000,  5.1000,  1.1000,  0.5000,  2.7000,  2.0000, 19.3000,
        16.4000,  0.5440,  0.3190])
	 x reshape =  tensor([19.3000])
	 x =  tensor([24.0000, 29.0000, 29.0000, 36.4000,  8

	 x reshape =  tensor([18.9000])
	 x =  tensor([20.0000, 27.0000, 27.0000, 33.5000,  5.6000, 13.8000,  0.4030,  2.4000,
         6.8000,  0.3500,  3.2000,  7.0000,  0.4550,  0.4890,  1.9000,  2.2000,
         0.8670,  4.7000,  5.4000,  0.4000,  0.1000,  2.4000,  2.4000, 15.4000,
        10.9000,  0.5220,  0.4310])
	 x reshape =  tensor([15.4000])
	 x =  tensor([26.0000,  8.0000,  4.0000, 23.6000,  4.3000, 10.8000,  0.3950,  1.9000,
         5.5000,  0.3410,  2.4000,  5.3000,  0.4520,  0.4830,  1.4000,  1.9000,
         0.7330,  1.8000,  3.9000,  1.0000,  1.0000,  0.9000,  2.3000, 11.8000,
        15.0000,  0.5080,  0.4580])
	 x reshape =  tensor([11.8000])
	 x =  tensor([31.0000, 20.0000,  0.0000, 14.8000,  3.5000,  5.7000,  0.6190,  0.0000,
         0.1000,  0.0000,  3.5000,  5.6000,  0.6250,  0.6190,  1.4000,  2.6000,
         0.5290,  5.7000,  0.5000,  0.3000,  1.4000,  1.0000,  1.8000,  8.4000,
        22.2000,  0.6170,  0.4310])
	 x reshape =  tensor([8.4000])
	 x =  tensor([25.00

In [729]:
wrong_i = [66,
            98,
            145,
            158,
            160,
            165,
            194,
            219,
            295,
            360,
            403,
            410,
            414,
            443,
            482,
            486]

index = 0
counter = 1;
for i, row in df1.iterrows():
    if index in wrong_i:
        print("index = ", index)
        print("wrong #", counter)
        print(row[0], row[24])
        print(" ")
        counter+=1
    index+=1

index =  66
wrong # 1
Malcolm Brogdon 21.6
 
index =  98
wrong # 2
Mike Conley 16.5
 
index =  145
wrong # 3
De'Aaron Fox 23.2
 
index =  158
wrong # 4
Shai Gilgeous-Alexander 22.5
 
index =  160
wrong # 5
Rudy Gobert 14.2
 
index =  165
wrong # 6
Jerami Grant 23.8
 
index =  194
wrong # 7
Gordon Hayward 22.3
 
index =  219
wrong # 8
Brandon Ingram 23.6
 
index =  295
wrong # 9
CJ McCollum 26.7
 
index =  360
wrong # 10
Chris Paul 17.2
 
index =  403
wrong # 11
Domantas Sabonis 21.5
 
index =  410
wrong # 12
Collin Sexton 22.8
 
index =  414
wrong # 13
Ben Simmons 15.2
 
index =  443
wrong # 14
Karl-Anthony Towns 21.8
 
index =  482
wrong # 15
Christian Wood 22.0
 
index =  486
wrong # 16
Trae Young 26.5
 


In [730]:
model_uni.evaluate(validation_vec, validation_lab)

	 x =  tensor([26.0000, 63.0000, 63.0000, 26.7000,  4.5000,  7.6000,  0.5920,  0.0000,
         0.0000,  0.3330,  4.5000,  7.5000,  0.5940,  0.5930,  1.9000,  3.2000,
         0.5820,  9.3000,  2.3000,  0.8000,  1.1000,  1.5000,  1.9000, 10.9000,
        20.5000,  0.6040,  0.6110])
	 x reshape =  tensor([10.9000])
	 x =  tensor([22.0000, 72.0000, 72.0000, 33.6000,  6.1000, 11.0000,  0.5570,  0.0000,
         0.2000,  0.1430,  6.1000, 10.8000,  0.5640,  0.5580,  3.7000,  5.3000,
         0.6910, 10.2000,  5.1000,  1.1000,  1.3000,  2.8000,  2.5000, 15.9000,
        20.3000,  0.5980,  0.6030])
	 x reshape =  tensor([15.9000])
index =  1
	 prediction, actual = 0, 1
	 x =  tensor([34.0000, 53.0000, 53.0000, 33.1000,  7.4000, 15.0000,  0.4930,  1.2000,
         3.0000,  0.3890,  6.2000, 12.0000,  0.5190,  0.5320,  3.0000,  3.6000,
         0.8270,  7.4000,  2.4000,  0.7000,  1.6000,  1.4000,  2.4000, 18.9000,
        19.7000,  0.5710,  0.4580])
	 x reshape =  tensor([18.9000])
	 x =  tensor

	 x =  tensor([19.0000, 61.0000,  9.0000, 18.5000,  2.0000,  5.2000,  0.3940,  0.8000,
         2.3000,  0.3480,  1.2000,  2.9000,  0.4320,  0.4720,  0.7000,  1.0000,
         0.6940,  4.0000,  0.7000,  0.4000,  0.7000,  0.7000,  0.9000,  5.6000,
         9.6000,  0.4970,  0.6110])
	 x reshape =  tensor([5.6000])
	 x =  tensor([26.0000, 57.0000, 57.0000, 36.0000, 10.4000, 22.9000,  0.4550,  3.0000,
         8.4000,  0.3530,  7.4000, 14.4000,  0.5150,  0.5200,  6.8000,  8.0000,
         0.8420,  4.2000,  6.1000,  1.2000,  0.4000,  3.4000,  2.2000, 30.5000,
        23.2000,  0.5790,  0.3470])
	 x reshape =  tensor([30.5000])
index =  37
	 prediction, actual = 1, 0
	 x =  tensor([23.0000, 41.0000,  0.0000, 18.2000,  2.9000,  7.3000,  0.3890,  1.4000,
         3.9000,  0.3600,  1.4000,  3.4000,  0.4210,  0.4850,  0.8000,  0.9000,
         0.8680,  1.9000,  1.2000,  0.8000,  0.1000,  0.9000,  1.2000,  7.9000,
        10.5000,  0.5110,  0.6300])
	 x reshape =  tensor([7.9000])
	 x =  tensor(

	 x =  tensor([23.0000, 57.0000, 57.0000, 33.9000,  7.5000, 15.6000,  0.4810,  2.3000,
         5.9000,  0.3820,  5.2000,  9.6000,  0.5430,  0.5540,  3.1000,  4.3000,
         0.7240,  6.4000,  2.1000,  1.1000,  0.4000,  2.2000,  2.9000, 20.3000,
        16.9000,  0.5830,  0.6670])
	 x reshape =  tensor([20.3000])
	 x =  tensor([20.0000,  9.0000,  0.0000,  3.7000,  0.4000,  1.1000,  0.4000,  0.0000,
         0.0000,  0.0000,  0.4000,  1.1000,  0.4000,  0.4000,  0.3000,  0.9000,
         0.3750,  1.6000,  0.1000,  0.1000,  0.1000,  0.3000,  0.8000,  1.2000,
         6.7000,  0.4070,  0.5830])
	 x reshape =  tensor([1.2000])
	 x =  tensor([24.0000, 52.0000,  1.0000, 14.8000,  1.8000,  4.9000,  0.3710,  0.8000,
         2.6000,  0.3240,  1.0000,  2.3000,  0.4250,  0.4570,  0.6000,  0.8000,
         0.8000,  3.5000,  1.0000,  0.6000,  0.1000,  0.8000,  1.0000,  5.1000,
         9.5000,  0.4860,  0.7670])
	 x reshape =  tensor([5.1000])
	 x =  tensor([20.0000, 69.0000, 22.0000, 25.8000,  4.

	 x =  tensor([26.0000, 41.0000, 37.0000, 22.9000,  3.5000,  6.3000,  0.5600,  0.0000,
         0.0000,  0.0000,  3.5000,  6.3000,  0.5600,  0.5600,  0.9000,  1.4000,
         0.6140,  6.2000,  1.5000,  1.1000,  1.2000,  0.9000,  2.5000,  7.9000,
        16.2000,  0.5730,  0.2310])
	 x reshape =  tensor([7.9000])
	 x =  tensor([26.0000, 13.0000,  2.0000, 12.1000,  2.4000,  3.5000,  0.6890,  0.0000,
         0.1000,  0.0000,  2.4000,  3.4000,  0.7050,  0.6890,  0.4000,  0.7000,
         0.5560,  4.6000,  0.8000,  0.3000,  0.8000,  0.4000,  1.5000,  5.2000,
        22.2000,  0.6840,  0.5730])
	 x reshape =  tensor([5.2000])
	 x =  tensor([37.0000, 26.0000,  5.0000,  8.4000,  0.5000,  0.7000,  0.7780,  0.0000,
         0.0000,  0.0000,  0.5000,  0.7000,  0.7780,  0.7780,  0.2000,  0.5000,
         0.4620,  2.5000,  0.2000,  0.2000,  0.3000,  0.3000,  1.2000,  1.3000,
        10.8000,  0.7170,  0.6110])
	 x reshape =  tensor([1.3000])
	 x =  tensor([32.0000, 35.0000,  3.0000, 21.0000,  2.1

	 x reshape =  tensor([9.9000])
	 x =  tensor([29.0000, 20.0000,  8.0000, 27.7000,  4.0000,  8.2000,  0.4820,  2.9000,
         6.4000,  0.4450,  1.1000,  1.8000,  0.6110,  0.6550,  1.1000,  1.5000,
         0.7330,  5.4000,  1.8000,  1.3000,  0.5000,  0.8000,  2.9000, 11.9000,
        16.3000,  0.6690,  0.6030])
	 x reshape =  tensor([11.9000])
	 x =  tensor([20.0000, 63.0000, 35.0000, 23.9000,  3.7000,  9.0000,  0.4040,  1.0000,
         3.5000,  0.2990,  2.6000,  5.5000,  0.4710,  0.4620,  0.9000,  1.9000,
         0.4620,  3.4000,  1.7000,  0.9000,  0.6000,  1.3000,  2.0000,  9.2000,
         9.6000,  0.4670,  0.2970])
	 x reshape =  tensor([9.2000])
	 x =  tensor([29.0000, 64.0000, 25.0000, 24.6000,  4.4000,  9.0000,  0.4950,  2.3000,
         5.0000,  0.4520,  2.2000,  4.0000,  0.5490,  0.6210,  1.3000,  1.5000,
         0.8250,  2.3000,  1.9000,  0.6000,  0.1000,  1.0000,  1.8000, 12.4000,
        15.5000,  0.6430,  0.5730])
	 x reshape =  tensor([12.4000])
	 x =  tensor([31.000

...

	 x =  tensor([27.0000, 68.0000,  4.0000, 23.4000,  3.1000,  7.4000,  0.4210,  1.4000,
         4.0000,  0.3560,  1.7000,  3.4000,  0.4980,  0.5170,  1.1000,  1.6000,
         0.7030,  2.6000,  1.7000,  0.7000,  0.1000,  0.7000,  1.9000,  8.8000,
        10.4000,  0.5420,  0.6110])
	 x reshape =  tensor([8.8000])
	 x =  tensor([28.0000,  7.0000,  0.0000, 12.4000,  1.1000,  4.1000,  0.2760,  0.4000,
         2.0000,  0.2140,  0.7000,  2.1000,  0.3330,  0.3280,  0.1000,  0.3000,
         0.5000,  3.9000,  0.6000,  0.1000,  0.4000,  0.3000,  1.9000,  2.9000,
         5.4000,  0.3350,  0.6110])
	 x reshape =  tensor([2.9000])
	 x =  tensor([25.0000, 73.0000, 68.0000, 29.7000,  4.4000,  9.4000,  0.4700,  3.7000,
         8.3000,  0.4460,  0.7000,  1.1000,  0.6540,  0.6670,  0.9000,  1.0000,
         0.9310,  3.2000,  1.4000,  0.5000,  0.3000,  1.0000,  2.6000, 13.5000,
        13.1000,  0.6840,  0.6030])
	 x reshape =  tensor([13.5000])
	 x =  tensor([26.0000, 48.0000, 48.0000, 31.6000,  5.

	 x reshape =  tensor([4.2000])
	 x =  tensor([25.0000, 60.0000, 60.0000, 35.2000,  8.3000, 18.4000,  0.4530,  2.2000,
         6.1000,  0.3590,  6.2000, 12.3000,  0.4990,  0.5120,  4.0000,  5.1000,
         0.7920,  7.3000,  3.5000,  1.0000,  0.9000,  2.5000,  2.8000, 22.9000,
        17.9000,  0.5540,  0.7360])
	 x reshape =  tensor([22.9000])
	 x =  tensor([23.0000, 44.0000,  0.0000,  7.9000,  1.1000,  1.8000,  0.6150,  0.0000,
         0.1000,  0.0000,  1.1000,  1.7000,  0.6400,  0.6150,  0.8000,  1.3000,
         0.6730,  2.9000,  0.5000,  0.2000,  0.5000,  0.7000,  1.9000,  3.0000,
        17.3000,  0.6510,  0.6030])
	 x reshape =  tensor([3.])
	 x =  tensor([23.0000, 57.0000, 57.0000, 35.4000,  6.6000, 11.4000,  0.5800,  0.0000,
         0.1000,  0.2860,  6.5000, 11.2000,  0.5830,  0.5810,  3.2000,  5.2000,
         0.6210,  7.8000,  8.0000,  2.1000,  0.6000,  3.5000,  3.3000, 16.4000,
        20.4000,  0.6020,  0.6810])
	 x reshape =  tensor([16.4000])
index =  499
	 prediction

	 x =  tensor([34.0000, 33.0000,  9.0000, 16.8000,  1.3000,  3.5000,  0.3680,  0.9000,
         2.7000,  0.3370,  0.4000,  0.8000,  0.4640,  0.4960,  0.4000,  0.6000,
         0.6840,  3.3000,  0.9000,  0.2000,  0.3000,  0.6000,  1.6000,  3.9000,
         7.5000,  0.5150,  0.5830])
	 x reshape =  tensor([3.9000])
	 x =  tensor([34.0000,  9.0000,  0.0000,  9.1000,  0.3000,  1.9000,  0.1760,  0.2000,
         1.7000,  0.1330,  0.1000,  0.2000,  0.5000,  0.2350,  0.1000,  0.2000,
         0.5000,  1.2000,  0.3000,  0.4000,  0.1000,  0.4000,  0.7000,  1.0000,
         0.2000,  0.2520,  0.4310])
	 x reshape =  tensor([1.])
	 x =  tensor([34.0000, 13.0000,  4.0000, 18.2000,  1.5000,  3.7000,  0.3960,  1.3000,
         3.2000,  0.4150,  0.2000,  0.5000,  0.2860,  0.5730,  0.5000,  0.6000,
         0.8750,  2.5000,  0.8000,  0.5000,  0.1000,  0.7000,  1.1000,  4.8000,
         8.0000,  0.6020,  0.4660])
	 x reshape =  tensor([4.8000])
	 x =  tensor([26.0000, 13.0000,  6.0000, 20.9000,  2.2000,

	 x reshape =  tensor([6.9000])
	 x =  tensor([26.0000, 67.0000, 67.0000, 32.9000,  8.0000, 14.9000,  0.5360,  1.4000,
         3.4000,  0.4030,  6.6000, 11.5000,  0.5750,  0.5810,  2.5000,  3.0000,
         0.8190,  4.2000,  1.5000,  1.2000,  0.5000,  1.3000,  2.8000, 19.8000,
        18.4000,  0.6100,  0.6160])
	 x reshape =  tensor([19.8000])
	 x =  tensor([21.0000, 58.0000, 57.0000, 30.3000,  4.6000, 10.2000,  0.4550,  1.5000,
         4.0000,  0.3740,  3.1000,  6.2000,  0.5070,  0.5280,  1.5000,  2.3000,
         0.6470,  5.4000,  2.1000,  0.9000,  0.8000,  1.6000,  2.7000, 12.2000,
        12.4000,  0.5470,  0.3540])
	 x reshape =  tensor([12.2000])
	 x =  tensor([25.0000, 18.0000,  0.0000,  5.8000,  0.8000,  1.9000,  0.4410,  0.2000,
         0.4000,  0.3750,  0.7000,  1.4000,  0.4620,  0.4850,  0.2000,  0.4000,
         0.3750,  1.1000,  0.3000,  0.3000,  0.1000,  0.1000,  0.5000,  2.0000,
        11.2000,  0.4800,  0.4660])
	 x reshape =  tensor([2.])
	 x =  tensor([22.0000, 1

	 x =  tensor([21.0000, 60.0000, 60.0000, 35.3000,  9.1000, 20.8000,  0.4370,  3.4000,
         9.5000,  0.3610,  5.7000, 11.4000,  0.5010,  0.5190,  8.0000,  9.3000,
         0.8600,  4.3000,  9.3000,  1.1000,  0.1000,  4.8000,  1.7000, 29.6000,
        23.9000,  0.5950,  0.2990])
	 x reshape =  tensor([29.6000])
	 x =  tensor([27.0000, 58.0000, 39.0000, 23.1000,  4.3000,  8.3000,  0.5240,  0.3000,
         1.3000,  0.2400,  4.0000,  7.0000,  0.5770,  0.5430,  2.1000,  3.1000,
         0.6820,  7.1000,  1.5000,  0.7000,  0.4000,  1.3000,  2.4000, 11.1000,
        18.8000,  0.5760,  0.3540])
	 x reshape =  tensor([11.1000])
	 x =  tensor([30.0000,  2.0000,  0.0000,  2.0000,  0.5000,  2.0000,  0.2500,  0.0000,
         0.0000,  0.0000,  0.5000,  2.0000,  0.2500,  0.2500,  0.0000,  0.0000,
         0.0000,  2.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  1.0000,
        22.4000,  0.2500,  0.4580])
	 x reshape =  tensor([1.])
	 x =  tensor([23.0000, 22.0000,  0.0000, 10.0000,  1.900

In [731]:
wrong_i = [1,
37,
61,
90,
122,
148,
202,
274,
297,
324,
342,
362,
377,
442,
482,
483,
484,
499,
529,
551,
569,
579]

index = 0
counter = 1;
for i, row in df2.iterrows():
    if index in wrong_i:
        print("index = ", index)
        print("wrong #", counter)
        print(row[0], row[24])
        print(" ")
        counter+=1
    index+=1

index =  1
wrong # 1
Bam Adebayo 15.9
 
index =  37
wrong # 2
Bradley Beal 30.5
 
index =  61
wrong # 3
Devin Booker 26.6
 
index =  90
wrong # 4
Jimmy Butler 19.9
 
index =  122
wrong # 5
John Collins 21.6
 
index =  148
wrong # 6
DeMar DeRozan 22.1
 
index =  202
wrong # 7
Rudy Gobert 15.1
 
index =  274
wrong # 8
Kyrie Irving 27.4
 
index =  297
wrong # 9
Nikola Jokic 19.9
 
index =  324
wrong # 10
Zach LaVine 25.5
 
index =  342
wrong # 11
Kyle Lowry 19.4
 
index =  362
wrong # 12
CJ McCollum 22.2
 
index =  377
wrong # 13
Khris Middleton 20.9
 
index =  442
wrong # 14
Chris Paul 17.6
 
index =  482
wrong # 15
D'Angelo Russell 23.6
 
index =  483
wrong # 16
D'Angelo Russell 21.7
 
index =  484
wrong # 17
Domantas Sabonis 18.5
 
index =  499
wrong # 18
Ben Simmons 16.4
 
index =  529
wrong # 19
Karl-Anthony Towns 26.5
 
index =  551
wrong # 20
Kemba Walker 20.4
 
index =  569
wrong # 21
Andrew Wiggins 22.4
 
index =  579
wrong # 22
Zion Williamson 22.5
 
